In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
pd.set_option('display.max_rows', None)

In [6]:
boe=pd.read_csv('../data/BOE_def.csv')

In [41]:
testeo=pd.read_csv('../data/BOE_Sin_Puja.csv')

testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   identificador         6154 non-null   object
 1   tipo_de_subasta       6154 non-null   object
 2   cuenta_expediente     6154 non-null   object
 3   fecha_de_inicio       6154 non-null   object
 4   fecha_de_conclusion   6154 non-null   object
 5   cantidad_reclamada    6154 non-null   int64 
 6   anuncio_BOE           6154 non-null   object
 7   valor_subasta         6154 non-null   int64 
 8   tasacion              6154 non-null   int64 
 9   puja_minima           6154 non-null   int64 
 10  tramos_entre_pujas    6154 non-null   int64 
 11  importe_del_deposito  6154 non-null   int64 
 12  direccion             6154 non-null   object
 13  codigo_postal         6154 non-null   int64 
 14  localidad             6154 non-null   object
 15  provincia             6154 non-null   

In [7]:
boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   identificador         6154 non-null   object
 1   tipo_de_subasta       6154 non-null   object
 2   cuenta_expediente     6154 non-null   object
 3   fecha_de_inicio       6154 non-null   object
 4   fecha_de_conclusion   6154 non-null   object
 5   cantidad_reclamada    6154 non-null   int64 
 6   anuncio_BOE           6154 non-null   object
 7   valor_subasta         6154 non-null   int64 
 8   tasacion              6154 non-null   int64 
 9   puja_minima           6154 non-null   int64 
 10  tramos_entre_pujas    6154 non-null   int64 
 11  importe_del_deposito  6154 non-null   int64 
 12  direccion             6154 non-null   object
 13  codigo_postal         6154 non-null   int64 
 14  localidad             6154 non-null   object
 15  provincia             6154 non-null   

In [8]:
import regex as re

In [9]:
def limpiar_(fecha_de_inicio):
    fecha_de_inicio=re.findall('\d{4}', fecha_de_inicio)
    fecha_de_inicio=' '.join(fecha_de_inicio)
    return fecha_de_inicio

In [10]:
boe.fecha_de_inicio=boe.fecha_de_inicio.apply(limpiar_)

In [42]:
testeo.fecha_de_inicio=testeo.fecha_de_inicio.apply(limpiar_)

In [11]:
boe.fecha_de_conclusion=boe.fecha_de_conclusion.apply(limpiar_)

In [43]:
testeo.fecha_de_conclusion=testeo.fecha_de_conclusion.apply(limpiar_)

In [12]:
boe.head()

,identificador,tipo_de_subasta,cuenta_expediente,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,anuncio_BOE,valor_subasta,tasacion,puja_minima,tramos_entre_pujas,importe_del_deposito,direccion,codigo_postal,localidad,provincia,puja,pm2,difference
0,SUB-JA-2022-204013,JUDICIAL EN VÍA DE APREMIO,4743 0000 05 1018 20,2022,2022,7145,BOE-B-2022-35270,108482,0,0,2169,5424,Urbanización Els Racons nº 5. partida Michafiga,46220,Picassent,Valencia/València,58580,1088,101337
1,SUB-JA-2022-196067,JUDICIAL EN VÍA DE APREMIO,2368 0000 06 0191 20,2022,2022,142298,BOE-B-2022-35269,305832,0,0,6116,15291,"CALLE GABRIEL GARCÍA MÁRQUEZ, 31",28980,PARLA,Madrid,177382,1721,163534
2,SUB-JA-2022-204060,JUDICIAL EN VÍA DE APREMIO,2995 0000 06 0456 16,2022,2022,55933,BOE-B-2022-35263,247902,0,0,0,12395,VIVIENDA 1 BLOQUE 19 CONJUNTO RESIDENCIAL HACI...,29600,Marbella,Málaga,252000,1721,191969
3,SUB-JA-2022-198972,JUDICIAL EN VÍA DE APREMIO,3997 0000 06 1661 20,2022,2022,210425,BOE-B-2022-28879,826272,0,0,16525,41313,"AVENIDA DE JACARANDA, 5, URBANIZACION CLUB DE ...",41930,BORMUJOS,Sevilla,512288,1557,615847
4,SUB-JA-2022-203383,JUDICIAL EN VÍA DE APREMIO,5418 0000 06 0169 21,2022,2022,34853,BOE-B-2022-35253,121684,121684,45978,2433,6084,"Calle Palma de Mallorca, nº 13, bloque 19, 3º ...",35016,Las Palmas de Gran Canaria,Las Palmas,45978,1130,86831


In [13]:
boe.drop(columns=['identificador','tipo_de_subasta','cuenta_expediente',
                  'anuncio_BOE','tasacion','puja_minima','tramos_entre_pujas',
                 'importe_del_deposito','direccion','localidad'], inplace=True)

In [44]:
testeo.drop(columns=['identificador','tipo_de_subasta','cuenta_expediente',
                  'anuncio_BOE','tasacion','puja_minima','tramos_entre_pujas',
                 'importe_del_deposito','direccion','localidad'], inplace=True)

In [47]:
testeo.head()

,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,pm2
0,2022,2022,7145,108482,46220,Valencia/València,1088
1,2022,2022,142298,305832,28980,Madrid,1721
2,2022,2022,55933,247902,29600,Málaga,1721
3,2022,2022,210425,826272,41930,Sevilla,1557
4,2022,2022,34853,121684,35016,Las Palmas,1130


In [14]:
boe.head()

,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2,difference
0,2022,2022,7145,108482,46220,Valencia/València,58580,1088,101337
1,2022,2022,142298,305832,28980,Madrid,177382,1721,163534
2,2022,2022,55933,247902,29600,Málaga,252000,1721,191969
3,2022,2022,210425,826272,41930,Sevilla,512288,1557,615847
4,2022,2022,34853,121684,35016,Las Palmas,45978,1130,86831


In [15]:
boe.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fecha_de_inicio      6154 non-null   object
 1   fecha_de_conclusion  6154 non-null   object
 2   cantidad_reclamada   6154 non-null   int64 
 3   valor_subasta        6154 non-null   int64 
 4   codigo_postal        6154 non-null   int64 
 5   provincia            6154 non-null   object
 6   puja                 6154 non-null   int64 
 7   pm2                  6154 non-null   int64 
 8   difference           6154 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 1.4 MB


In [16]:
boe['fecha_de_inicio'] = boe['fecha_de_inicio'].astype(int)

In [48]:
testeo['fecha_de_inicio'] = testeo['fecha_de_inicio'].astype(int)

In [17]:
boe['fecha_de_conclusion'] = boe['fecha_de_conclusion'].astype(int)

In [49]:
testeo['fecha_de_conclusion'] = testeo['fecha_de_conclusion'].astype(int)

In [18]:
#eliminamos esta columna porque el precio se cierra en la fecha de conclusion y varia segun el año final.
boe.drop(columns=['fecha_de_inicio'], inplace=True)

In [50]:
#eliminamos esta columna porque el precio se cierra en la fecha de conclusion y varia segun el año final.
testeo.drop(columns=['fecha_de_inicio'], inplace=True)

In [19]:
#eliminamos codigo postal? con eso hemos sacado el pm2, no nos interesa más. Ademas contamos con la provincia.
#boe.drop(columns=['codigo_postal'], inplace=True)

In [51]:
#creamos new colum con la diferencia entre la deuda(cantidad_reclamada) y el valor que le da el juez(valor_subasta)
#testeo['difference'] = testeo['valor_subasta'].sub(testeo['cantidad_reclamada'], axis = 0)

In [52]:
testeo.head()

,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,pm2,difference
0,2022,7145,108482,46220,Valencia/València,1088,101337
1,2022,142298,305832,28980,Madrid,1721,163534
2,2022,55933,247902,29600,Málaga,1721,191969
3,2022,210425,826272,41930,Sevilla,1557,615847
4,2022,34853,121684,35016,Las Palmas,1130,86831


In [20]:
boe.head()

,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2,difference
0,2022,7145,108482,46220,Valencia/València,58580,1088,101337
1,2022,142298,305832,28980,Madrid,177382,1721,163534
2,2022,55933,247902,29600,Málaga,252000,1721,191969
3,2022,210425,826272,41930,Sevilla,512288,1557,615847
4,2022,34853,121684,35016,Las Palmas,45978,1130,86831


In [ ]:
#boe.drop(columns=['cantidad_reclamada','valor_subasta'], inplace=True)

In [21]:
boe=pd.get_dummies(boe, columns=['fecha_de_conclusion','provincia','codigo_postal'], drop_first=True)

boe.head()

cantidad_reclamada  valor_subasta    puja   pm2  difference  \
0                7145         108482   58580  1088      101337   
1              142298         305832  177382  1721      163534   
2               55933         247902  252000  1721      191969   
3              210425         826272  512288  1557      615847   
4               34853         121684   45978  1130       86831   

   fecha_de_conclusion_2017  fecha_de_conclusion_2018  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2019  fecha_de_conclusion_2020  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2021  fecha_de_conclusion_2022  provincia_Albacete  \
0                         0                         1                   0   
1                         0                         1                   0   
2                         0                         1                   0   
3                         0                         1                   0   
4                         0                         1                   0   

   provincia_Alicante  provincia_Alicante/Alacant  provincia_Almería  \
0                   0                           0                  0   
1                   0                           0                  0   
2                   0                           0                  0   
3                   0                           0                  0   
4                   0                           0                  0   

   provincia_Araba/Álava  provincia_Asturias  provincia_Badajoz  \
0                      0                   0                  0   
1                      0                   0                  0   
2                      0                   0                  0   
3                      0                   0                  0   
4                      0                   0                  0   

   provincia_Baleares  provincia_Barcelona  provincia_Bizkaia  \
0                   0                    0                  0   
1                   0                    0                  0   
2                   0                    0                  0   
3                   0                    0                  0   
4                   0                    0                  0   

   provincia_Burgos  provincia_Cantabria  provincia_Castellón  \
0                 0                    0                    0   
1                 0                    0                    0   
2                 0                    0                    0   
3                 0                    0                    0   
4                 0                    0                    0   

   provincia_Castellón/Castelló  provincia_Ceuta  provincia_Ciudad Real  \
0                             0                0                      0   
1                             0                0                      0   
2                             0                0                      0   
3                             0                0                      0   
4                             0                0                      0   

   provincia_Cuenca  provincia_Cáceres  provincia_Cádiz  provincia_Córdoba  \
0                 0                  0                0                  0   
1                 0                  0                0                  0   
2                 0                  0                0                  0   
3                 0                  0                0                  0  

In [53]:
testeo=pd.get_dummies(testeo, columns=['fecha_de_conclusion','provincia','codigo_postal'], drop_first=True)

testeo.head()

cantidad_reclamada  valor_subasta   pm2  difference  \
0                7145         108482  1088      101337   
1              142298         305832  1721      163534   
2               55933         247902  1721      191969   
3              210425         826272  1557      615847   
4               34853         121684  1130       86831   

   fecha_de_conclusion_2017  fecha_de_conclusion_2018  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2019  fecha_de_conclusion_2020  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2021  fecha_de_conclusion_2022  provincia_Albacete  \
0                         0                         1                   0   
1                         0                         1                   0   
2                         0                         1                   0   
3                         0                         1                   0   
4                         0                         1                   0   

   provincia_Alicante  provincia_Alicante/Alacant  provincia_Almería  \
0                   0                           0                  0   
1                   0                           0                  0   
2                   0                           0                  0   
3                   0                           0                  0   
4                   0                           0                  0   

   provincia_Araba/Álava  provincia_Asturias  provincia_Badajoz  \
0                      0                   0                  0   
1                      0                   0                  0   
2                      0                   0                  0   
3                      0                   0                  0   
4                      0                   0                  0   

   provincia_Baleares  provincia_Barcelona  provincia_Bizkaia  \
0                   0                    0                  0   
1                   0                    0                  0   
2                   0                    0                  0   
3                   0                    0                  0   
4                   0                    0                  0   

   provincia_Burgos  provincia_Cantabria  provincia_Castellón  \
0                 0                    0                    0   
1                 0                    0                    0   
2                 0                    0                    0   
3                 0                    0                    0   
4                 0                    0                    0   

   provincia_Castellón/Castelló  provincia_Ceuta  provincia_Ciudad Real  \
0                             0                0                      0   
1                             0                0                      0   
2                             0                0                      0   
3                             0                0                      0   
4                             0                0                      0   

   provincia_Cuenca  provincia_Cáceres  provincia_Cádiz  provincia_Córdoba  \
0                 0                  0                0                  0   
1                 0                  0                0                  0   
2                 0                  0                0                  0   
3                 0                  0                0                  0   
4                 0                  0        

In [22]:
boe.pm2=LabelEncoder().fit_transform(boe.pm2)

In [23]:
boe.cantidad_reclamada=LabelEncoder().fit_transform(boe.cantidad_reclamada)

In [24]:
boe.valor_subasta=LabelEncoder().fit_transform(boe.valor_subasta)

In [25]:
boe.difference=LabelEncoder().fit_transform(boe.difference)

In [54]:
testeo.pm2=LabelEncoder().fit_transform(testeo.pm2)

In [55]:
testeo.cantidad_reclamada=LabelEncoder().fit_transform(testeo.cantidad_reclamada)

In [56]:
testeo.valor_subasta=LabelEncoder().fit_transform(testeo.valor_subasta)

In [57]:
testeo.difference=LabelEncoder().fit_transform(testeo.difference)

In [26]:
#boe=boe.rename(columns = {'diff':'diference'})

In [27]:
boe_x=boe.drop('puja', axis=1)
boe_y=boe.puja

In [28]:
X_train, X_test, y_train, y_test = tts(boe_x, boe_y,train_size=0.9, test_size=0.1,random_state =71)

**Iniciamos los modelos**

In [29]:
linreg=LinReg()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
svr=SVR()
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
lgbmr=LGBMR()

**Entrenamos modelos**

In [30]:
linreg.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic.fit(X_train, y_train)
svr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)

Learning rate set to 0.053657
0:	learn: 584915.1604973	total: 64.6ms	remaining: 1m 4s
1:	learn: 578347.3759690	total: 67.1ms	remaining: 33.5s
2:	learn: 571884.1289702	total: 69.8ms	remaining: 23.2s
3:	learn: 565747.4920656	total: 71ms	remaining: 17.7s
4:	learn: 559617.8710251	total: 73.4ms	remaining: 14.6s
5:	learn: 553670.8190351	total: 76ms	remaining: 12.6s
6:	learn: 547819.6953067	total: 79ms	remaining: 11.2s
7:	learn: 542011.8769577	total: 82.1ms	remaining: 10.2s
8:	learn: 536352.3865506	total: 85ms	remaining: 9.36s
9:	learn: 530725.0263818	total: 87.9ms	remaining: 8.7s
10:	learn: 525070.8698431	total: 90.9ms	remaining: 8.17s
11:	learn: 519600.5681040	total: 93.6ms	remaining: 7.7s
12:	learn: 514262.8863618	total: 96.3ms	remaining: 7.31s
13:	learn: 509019.0730918	total: 99.3ms	remaining: 6.99s
14:	learn: 504956.9174998	total: 102ms	remaining: 6.71s
15:	learn: 499920.0928444	total: 105ms	remaining: 6.45s
16:	learn: 494950.9758853	total: 108ms	remaining: 6.23s
17:	learn: 489933.496621

184:	learn: 124361.7949822	total: 589ms	remaining: 2.59s
185:	learn: 123405.9252369	total: 592ms	remaining: 2.59s
186:	learn: 122571.9594571	total: 595ms	remaining: 2.59s
187:	learn: 121675.3108296	total: 598ms	remaining: 2.58s
188:	learn: 120978.0258684	total: 601ms	remaining: 2.58s
189:	learn: 120293.1488708	total: 604ms	remaining: 2.57s
190:	learn: 119626.6011798	total: 607ms	remaining: 2.57s
191:	learn: 118877.5499319	total: 610ms	remaining: 2.57s
192:	learn: 118212.6829012	total: 613ms	remaining: 2.56s
193:	learn: 117590.3074198	total: 616ms	remaining: 2.56s
194:	learn: 116792.1815309	total: 619ms	remaining: 2.55s
195:	learn: 116185.5971414	total: 621ms	remaining: 2.55s
196:	learn: 115577.4834678	total: 624ms	remaining: 2.54s
197:	learn: 115005.9233446	total: 627ms	remaining: 2.54s
198:	learn: 114213.3492201	total: 630ms	remaining: 2.54s
199:	learn: 113423.3011972	total: 633ms	remaining: 2.53s
200:	learn: 113039.2528607	total: 636ms	remaining: 2.53s
201:	learn: 112583.6098706	tota

390:	learn: 71747.8766497	total: 1.19s	remaining: 1.85s
391:	learn: 71590.9335783	total: 1.2s	remaining: 1.85s
392:	learn: 71460.0013719	total: 1.2s	remaining: 1.85s
393:	learn: 71432.7360002	total: 1.2s	remaining: 1.85s
394:	learn: 71375.3357550	total: 1.21s	remaining: 1.85s
395:	learn: 71348.6961516	total: 1.21s	remaining: 1.84s
396:	learn: 71292.9692375	total: 1.21s	remaining: 1.84s
397:	learn: 71236.9833940	total: 1.22s	remaining: 1.84s
398:	learn: 71210.9002928	total: 1.23s	remaining: 1.85s
399:	learn: 71157.2293909	total: 1.23s	remaining: 1.84s
400:	learn: 71132.9009655	total: 1.23s	remaining: 1.84s
401:	learn: 71012.6823765	total: 1.24s	remaining: 1.84s
402:	learn: 70883.2965043	total: 1.24s	remaining: 1.84s
403:	learn: 70739.5588992	total: 1.24s	remaining: 1.83s
404:	learn: 70639.3861716	total: 1.25s	remaining: 1.83s
405:	learn: 70614.3438916	total: 1.25s	remaining: 1.83s
406:	learn: 70566.4079339	total: 1.25s	remaining: 1.83s
407:	learn: 70473.2811286	total: 1.26s	remaining: 1

590:	learn: 61929.1536248	total: 1.78s	remaining: 1.23s
591:	learn: 61917.1964124	total: 1.78s	remaining: 1.23s
592:	learn: 61905.8663438	total: 1.79s	remaining: 1.23s
593:	learn: 61834.4300250	total: 1.79s	remaining: 1.22s
594:	learn: 61789.6334693	total: 1.79s	remaining: 1.22s
595:	learn: 61770.4805559	total: 1.8s	remaining: 1.22s
596:	learn: 61751.9038591	total: 1.8s	remaining: 1.21s
597:	learn: 61733.8814980	total: 1.8s	remaining: 1.21s
598:	learn: 61722.0819510	total: 1.8s	remaining: 1.21s
599:	learn: 61618.2714136	total: 1.81s	remaining: 1.2s
600:	learn: 61597.2887444	total: 1.81s	remaining: 1.2s
601:	learn: 61586.0585589	total: 1.81s	remaining: 1.2s
602:	learn: 61568.2752023	total: 1.82s	remaining: 1.2s
603:	learn: 61557.1350996	total: 1.82s	remaining: 1.19s
604:	learn: 61539.3052557	total: 1.82s	remaining: 1.19s
605:	learn: 61521.3268289	total: 1.82s	remaining: 1.19s
606:	learn: 61500.7614103	total: 1.83s	remaining: 1.18s
607:	learn: 61483.0523243	total: 1.83s	remaining: 1.18s


796:	learn: 56482.4108448	total: 2.38s	remaining: 605ms
797:	learn: 56470.3444778	total: 2.38s	remaining: 602ms
798:	learn: 56452.4808437	total: 2.38s	remaining: 599ms
799:	learn: 56408.6525750	total: 2.38s	remaining: 596ms
800:	learn: 56370.0739223	total: 2.39s	remaining: 593ms
801:	learn: 56361.9214536	total: 2.39s	remaining: 590ms
802:	learn: 56326.5366018	total: 2.39s	remaining: 587ms
803:	learn: 56292.1849407	total: 2.4s	remaining: 584ms
804:	learn: 56284.2393185	total: 2.4s	remaining: 581ms
805:	learn: 56276.9881767	total: 2.4s	remaining: 578ms
806:	learn: 56246.7557535	total: 2.4s	remaining: 575ms
807:	learn: 56239.5302078	total: 2.41s	remaining: 572ms
808:	learn: 56227.5958824	total: 2.41s	remaining: 569ms
809:	learn: 56203.1766855	total: 2.41s	remaining: 566ms
810:	learn: 56170.1916250	total: 2.42s	remaining: 564ms
811:	learn: 56126.5280167	total: 2.42s	remaining: 561ms
812:	learn: 56119.3156663	total: 2.42s	remaining: 557ms
813:	learn: 56072.2875220	total: 2.43s	remaining: 55

LGBMRegressor()

In [31]:
from sklearn.metrics import mean_squared_error as mse

In [32]:
from sklearn.metrics import r2_score as r2

In [33]:
# regresion lineal

y_pred=linreg.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(711455.5924279436, -1.0760358932401424)

In [34]:
linreg.score(X_train, y_train), linreg.score(X_test, y_test)


(0.35887764254395693, -1.0760358932401424)

In [35]:
# ridge

y_pred=ridge.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(619396.5996483864, -0.5735367725898042)

In [36]:
## rfr

y_pred=rfr.predict(X_test)

mse(y_test, y_pred, squared=False)  , r2(y_test, y_pred)# RMSE

(294789.17886155675, 0.6435797769759306)

In [37]:
##gbr
y_pred=gbr.predict(X_test)

mse(y_test, y_pred, squared=False) , r2(y_test, y_pred) 

(306895.53584170784, 0.6137038274241035)

In [38]:
##xgbr

y_pred=xgbr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(1131163.8397465458, -4.247957264141792)

In [39]:
##ctr
y_pred=ctr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(376866.0476294013, 0.41747671634422845)

In [40]:
## lbbmr 87%
y_pred=lgbmr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(175773.85504769185, 0.8732792255858954)

In [58]:
y_pred=lgbmr.predict(testeo)

In [64]:
type(y_pred)

numpy.ndarray

In [85]:
type(prediccion1)

bytes

In [86]:
prediccion=pd.DataFrame(y_pred)

In [79]:
boe.head()

cantidad_reclamada  valor_subasta    puja  pm2  difference  \
0                 188           1636   58580  436        3988   
1                3822           4489  177382  805        4951   
2                1587           3943  252000  805        5186   
3                4765           5392  512288  715        5790   
4                 919           1910   45978  467        3602   

   fecha_de_conclusion_2017  fecha_de_conclusion_2018  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2019  fecha_de_conclusion_2020  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2021  fecha_de_conclusion_2022  provincia_Albacete  \
0                         0                         1                   0   
1                         0                         1                   0   
2                         0                         1                   0   
3                         0                         1                   0   
4                         0                         1                   0   

   provincia_Alicante  provincia_Alicante/Alacant  provincia_Almería  \
0                   0                           0                  0   
1                   0                           0                  0   
2                   0                           0                  0   
3                   0                           0                  0   
4                   0                           0                  0   

   provincia_Araba/Álava  provincia_Asturias  provincia_Badajoz  \
0                      0                   0                  0   
1                      0                   0                  0   
2                      0                   0                  0   
3                      0                   0                  0   
4                      0                   0                  0   

   provincia_Baleares  provincia_Barcelona  provincia_Bizkaia  \
0                   0                    0                  0   
1                   0                    0                  0   
2                   0                    0                  0   
3                   0                    0                  0   
4                   0                    0                  0   

   provincia_Burgos  provincia_Cantabria  provincia_Castellón  \
0                 0                    0                    0   
1                 0                    0                    0   
2                 0                    0                    0   
3                 0                    0                    0   
4                 0                    0                    0   

   provincia_Castellón/Castelló  provincia_Ceuta  provincia_Ciudad Real  \
0                             0                0                      0   
1                             0                0                      0   
2                             0                0                      0   
3                             0                0                      0   
4                             0                0                      0   

   provincia_Cuenca  provincia_Cáceres  provincia_Cádiz  provincia_Córdoba  \
0                 0                  0                0                  0   
1                 0                  0                0                  0   
2                 0                  0                0                  0   
3                 0                  0                0                  0   
4   

In [90]:
prediccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   prediccion_puja  6154 non-null   float64
dtypes: float64(1)
memory usage: 48.2 KB


In [89]:
prediccion.columns = ['prediccion_puja']

In [91]:
prediccion['prediccion_puja'] = prediccion['prediccion_puja'].astype(str)

In [92]:
prediccion['prediccion_puja'] = prediccion['prediccion_puja'].str.split('.').str[0]

In [96]:
prediccion.head()

,prediccion_puja
0,53591
1,159333
2,130042
3,400756
4,62614


In [99]:
boe=pd.read_csv('../data/BOE_def.csv')

In [100]:
boe = boe.join(prediccion)

In [101]:
boe.head()

,identificador,tipo_de_subasta,cuenta_expediente,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,anuncio_BOE,valor_subasta,tasacion,puja_minima,tramos_entre_pujas,importe_del_deposito,direccion,codigo_postal,localidad,provincia,puja,pm2,difference,prediccion_puja
0,SUB-JA-2022-204013,JUDICIAL EN VÍA DE APREMIO,4743 0000 05 1018 20,17-11-2022,08-12-2022,7145,BOE-B-2022-35270,108482,0,0,2169,5424,Urbanización Els Racons nº 5. partida Michafiga,46220,Picassent,Valencia/València,58580,1088,101337,53591
1,SUB-JA-2022-196067,JUDICIAL EN VÍA DE APREMIO,2368 0000 06 0191 20,17-11-2022,08-12-2022,142298,BOE-B-2022-35269,305832,0,0,6116,15291,"CALLE GABRIEL GARCÍA MÁRQUEZ, 31",28980,PARLA,Madrid,177382,1721,163534,159333
2,SUB-JA-2022-204060,JUDICIAL EN VÍA DE APREMIO,2995 0000 06 0456 16,17-11-2022,08-12-2022,55933,BOE-B-2022-35263,247902,0,0,0,12395,VIVIENDA 1 BLOQUE 19 CONJUNTO RESIDENCIAL HACI...,29600,Marbella,Málaga,252000,1721,191969,130042
3,SUB-JA-2022-198972,JUDICIAL EN VÍA DE APREMIO,3997 0000 06 1661 20,27-09-2022,08-12-2022,210425,BOE-B-2022-28879,826272,0,0,16525,41313,"AVENIDA DE JACARANDA, 5, URBANIZACION CLUB DE ...",41930,BORMUJOS,Sevilla,512288,1557,615847,400756
4,SUB-JA-2022-203383,JUDICIAL EN VÍA DE APREMIO,5418 0000 06 0169 21,17-11-2022,07-12-2022,34853,BOE-B-2022-35253,121684,121684,45978,2433,6084,"Calle Palma de Mallorca, nº 13, bloque 19, 3º ...",35016,Las Palmas de Gran Canaria,Las Palmas,45978,1130,86831,62614


In [102]:
boe.to_csv('../data/BOE_definitivo.csv', index=False, sep=',')

In [103]:
boe.to_excel('../data/BOE_definitivo.xlsx', index=False)